# Testing

In [1]:
import ib_async
print(ib_async.__all__)

['Event', 'util', 'Client', 'Bag', 'Bond', 'CFD', 'ComboLeg', 'Commodity', 'ContFuture', 'Contract', 'ContractDescription', 'ContractDetails', 'Crypto', 'DeltaNeutralContract', 'Forex', 'Future', 'FuturesOption', 'Index', 'MutualFund', 'Option', 'ScanData', 'Stock', 'TagValue', 'Warrant', 'FlexError', 'FlexReport', 'IB', 'IBC', 'Watchdog', 'AccountValue', 'BarData', 'BarDataList', 'CommissionReport', 'ConnectionStats', 'DOMLevel', 'DepthMktDataDescription', 'Dividends', 'Execution', 'ExecutionFilter', 'FamilyCode', 'Fill', 'FundamentalRatios', 'HistogramData', 'HistoricalNews', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'HistoricalSchedule', 'HistoricalSession', 'MktDepthData', 'NewsArticle', 'NewsBulletin', 'NewsProvider', 'NewsTick', 'OptionChain', 'OptionComputation', 'PnL', 'PnLSingle', 'PortfolioItem', 'Position', 'PriceIncrement', 'RealTimeBar', 'RealTimeBarList', 'ScanDataList', 'ScannerSubscription', 'SmartComponent', 'SoftDollarTier', 'TickAttrib', 'TickAt

### Importing
The following two lines are used at the top of all notebooks. The first line imports everything and the second
starts an event loop to keep the notebook live updated:

In [2]:
from ib_async import *
from ib_async.order import LimitOrder
from ibkr.broker.sim_ib import IBSim
from ibkr.broker.sim_contract_sim import load_contract
from ibkr.broker.sim_contract_sim import load_csv

util.startLoop()

*Note that startLoop() only works in notebooks, not in regular Python programs.*

### Connecting
The main player of the whole package is the "IB" class. Let's create an IB instance and connect to a running TWS/IBG application:

In [3]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=10)

<IB connected to 127.0.0.1:7497 clientId=10>

Error 478, reqId 37: Parameters in request conflicts with contract parameters received by contract id: requested expiry 20241221, in contract 20241220;
Canceled order: Trade(contract=Future(conId=495512557, symbol='ES', lastTradeDateOrContractMonth='20241221', multiplier='50', exchange='CME', currency='USD', localSymbol='ESZ4', tradingClass='ES'), order=LimitOrder(orderId=37, clientId=10, action='BUY', totalQuantity=1, lmtPrice=5853.25), orderStatus=OrderStatus(orderId=37, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 10, 12, 5, 40, 23, 726348, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 10, 12, 5, 40, 23, 733034, tzinfo=datetime.timezone.utc), status='Cancelled', message='Error 478, reqId 37: Parameters in request conflicts with contract parameters 

In [4]:
ib.accountSummary()

[AccountValue(account='DU1215439', tag='AccountType', value='INDIVIDUAL', currency='', modelCode=''),
 AccountValue(account='DU1215439', tag='Cushion', value='1', currency='', modelCode=''),
 AccountValue(account='DU1215439', tag='LookAheadNextChange', value='0', currency='', modelCode=''),
 AccountValue(account='DU1215439', tag='AccruedCash', value='81.21', currency='AUD', modelCode=''),
 AccountValue(account='DU1215439', tag='AvailableFunds', value='101425.97', currency='AUD', modelCode=''),
 AccountValue(account='DU1215439', tag='BuyingPower', value='101344.76', currency='AUD', modelCode=''),
 AccountValue(account='DU1215439', tag='EquityWithLoanValue', value='101425.97', currency='AUD', modelCode=''),
 AccountValue(account='DU1215439', tag='ExcessLiquidity', value='101425.97', currency='AUD', modelCode=''),
 AccountValue(account='DU1215439', tag='FullAvailableFunds', value='101425.97', currency='AUD', modelCode=''),
 AccountValue(account='DU1215439', tag='FullExcessLiquidity', valu

In [5]:
ib.accountValues()

[AccountValue(account='DU1215439', tag='AccountCode', value='DU1215439', currency='', modelCode=''),
 AccountValue(account='DU1215439', tag='AccountOrGroup', value='DU1215439', currency='AUD', modelCode=''),
 AccountValue(account='DU1215439', tag='AccountOrGroup', value='DU1215439', currency='BASE', modelCode=''),
 AccountValue(account='DU1215439', tag='AccountOrGroup', value='DU1215439', currency='USD', modelCode=''),
 AccountValue(account='DU1215439', tag='AccountReady', value='true', currency='', modelCode=''),
 AccountValue(account='DU1215439', tag='AccountType', value='INDIVIDUAL', currency='', modelCode=''),
 AccountValue(account='DU1215439', tag='AccruedCash', value='81.21', currency='AUD', modelCode=''),
 AccountValue(account='DU1215439', tag='AccruedCash', value='81.21', currency='BASE', modelCode=''),
 AccountValue(account='DU1215439', tag='AccruedCash', value='0.00', currency='USD', modelCode=''),
 AccountValue(account='DU1215439', tag='AccruedCash-P', value='0.00', currency

In [6]:
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DU1215439', tag='NetLiquidationByCurrency', value='101425.9663', currency='BASE', modelCode='')]

### Contracts

Contracts can be specified in different ways:
* The ibapi way, by creating an empty Contract object and setting its attributes one by one;
* By using Contract and giving the attributes as keyword argument;
* By using the specialized Stock, Option, Future, Forex, Index, CFD, Commodity,
  Bond, FuturesOption, MutualFund or Warrant contracts.

Some examples:

In [7]:
# contract = load_contract('ES')
contract = Future(symbol='ES', lastTradeDateOrContractMonth='202412', exchange='CME')
ib.qualifyContracts(contract)
contract

Future(conId=495512557, symbol='ES', lastTradeDateOrContractMonth='20241221', multiplier='50', exchange='CME', currency='USD', localSymbol='ESZ4', tradingClass='ES')

In [15]:
cds = ib.reqContractDetails(contract)
cds

[ContractDetails(contract=Contract(secType='FUT', conId=495512557, symbol='ES', lastTradeDateOrContractMonth='20241221', multiplier='50', exchange='CME', currency='USD', localSymbol='ESZ4', tradingClass='ES'), marketName='ES', minTick=0.25, orderTypes='ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASKET,BENCHPX,COND,CONDORDER,DAY,DEACT,DEACTDIS,DEACTEOD,GAT,GTC,GTD,GTT,HID,ICE,IOC,LIT,LMT,LTH,MIT,MKT,MTL,NGCOMB,NONALGO,OCA,PEGBENCH,SCALE,SCALERST,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='CME,QBALGO', priceMagnifier=1, underConId=11004968, longName='E-mini S&P 500', contractMonth='202412', industry='', category='', subcategory='', timeZoneId='Australia/Melbourne', tradingHours='20241012:CLOSED;20241013:1700-20241014:1600;20241014:1700-20241015:1600;20241015:1700-20241016:1600;20241016:1700-20241017:1600;20241017:1700-20241018:1600', liquidHours='20241012:CLOSED;20241013:CLOSED;20241014:0830-20241014:1600;20241015:0830-20241015:1600;2024

In [9]:
contracts = [cd.contract for cd in cds]

contracts

[Contract(secType='FUT', conId=495512557, symbol='ES', lastTradeDateOrContractMonth='20241221', multiplier='50', exchange='CME', currency='USD', localSymbol='ESZ4', tradingClass='ES')]

In [10]:
ib.reqHeadTimeStamp(contract, whatToShow='TRADES', useRTH=True)

datetime.datetime(2021, 8, 11, 22, 0)

Create a contract and a market order:

In [11]:
order = LimitOrder('BUY', 1, 5853.25)

In [12]:
trade = ib.placeOrder(contract, order)

In [13]:
ib.sleep(1)
trade.log

[TradeLogEntry(time=datetime.datetime(2024, 10, 12, 5, 40, 23, 726348, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2024, 10, 12, 5, 40, 23, 733034, tzinfo=datetime.timezone.utc), status='Cancelled', message='Error 478, reqId 37: Parameters in request conflicts with contract parameters received by contract id: requested expiry 20241221, in contract 20241220;', errorCode=478)]

In [14]:
ib.openOrders()

[]

In [44]:
trade.isDone()

True

In [45]:
ib.positions()

[]

In [20]:
contract

Future(symbol='ES', exchange='GLOBEX')

In [7]:
ib.trades()

[]

In [8]:
ib.openTrades()

[]

In [9]:
ib.fills()

[]

In [10]:
ib.reqOpenOrders()

[]

In [11]:
ib.reqAllOpenOrders()

[]

In [13]:
ib.reqCompletedOrders(apiOnly=True)

[]

In [6]:
for order in ib.orders():
    ib.cancelOrder(order)
